# Export heat stress indices
- This script is used to export heat stress indices from simulations;
- Simulations: CNTL, TRAF at UK-Manchester. 

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
home_path = '/gws/nopw/j04/duicv/yuansun/'

In [2]:
archive = f'{home_path}0_urban_traffic/archive'
start_date = '2012-01-01T00:00:00'
end_date = '2012-12-31T23:00:00'
case_list = ['cntl', 'traffic'] 
GRIDNAME_single = 'UK-MCR'   
var_list = ['HIA_U', 'SWBGT_U', 'DISCOIS_U', 'TSA_U', 'RH2M', 'TG_U'] # 'APPAR_TEMP_U'

In [3]:
df_all = pd.DataFrame()
for case in case_list:
    ds_case = xr.open_dataset(f'{archive}/{GRIDNAME_single}_{case}/lnd/hist/{GRIDNAME_single}_{case}.clm2.h0.2022-01-01-03600.nc')
    for var in var_list:   
        ds_case_var = ds_case[var] 
        df_case_var=ds_case_var.to_dataframe().reset_index()[['time', var]]
        df_case_var.rename(columns={var: f'{var}_{case}'}, inplace=True)
        df_case_var['time'] = df_case_var['time'].dt.round('min').dt.ceil('min')
        df_case_var['time'] = pd.to_datetime(df_case_var['time'])
        if df_all.empty:
            df_all = df_case_var
        else:
            df_all = pd.merge(df_all, df_case_var, how='outer', on='time')    
df_all['time'] = df_all['time'] - pd.Timedelta(hours=1)     
df_all.to_csv('./data_for_figure/heat_stress.csv', index=False)  
df_all.head()       

,time,HIA_U_cntl,SWBGT_U_cntl,DISCOIS_U_cntl,TSA_U_cntl,RH2M_cntl,TG_U_cntl,HIA_U_traffic,SWBGT_U_traffic,DISCOIS_U_traffic,TSA_U_traffic,RH2M_traffic,TG_U_traffic
0,2022-01-01 00:00:00,13.507699,16.903126,12.787918,286.657684,87.172379,285.816711,13.507699,16.903126,12.787918,286.657684,87.172379,285.866669
1,2022-01-01 01:00:00,13.702210,17.060009,12.961792,286.852203,86.831398,285.811066,13.743844,17.092461,12.997971,286.893829,86.739693,285.883789
2,2022-01-01 02:00:00,13.773051,17.084122,12.996030,286.923065,86.172905,285.525574,13.818438,17.118382,13.034526,286.968445,86.056313,285.586761
3,2022-01-01 03:00:00,13.429821,16.685949,12.580742,286.579834,84.758057,285.159454,13.458423,16.707151,12.604872,286.608429,84.682343,285.215820
4,2022-01-01 04:00:00,13.139027,16.360123,12.236031,286.289032,83.672836,284.887543,13.165053,16.379429,12.258155,286.315063,83.607140,284.941284
